In [1]:
import xarray as xr
import numpy as np
import scipy as sp

In [9]:
folder = r"C:\Users\joag\Documents\Notebooks"
path_timeseriesdata_emc3 = folder  + r"\N01.nc"
path_timeseriesdata_emc3
ds_tot = xr.open_dataset(path_timeseriesdata_emc3)

In [13]:
ds_tsdata_emc3 = ds_tot.isel(time = slice(-20,None))
mean_sel = ds_tsdata_emc3.mean(dim = 'time')
std_sel = ds_tsdata_emc3.std(dim = 'time')
std_tot = ds_tsdata_emc3.std()

C:\Users\joag\AppData\Local\Continuum\anaconda3\envs\Mayavi\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\joag\AppData\Local\Continuum\anaconda3\envs\Mayavi\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\joag\AppData\Local\Continuum\anaconda3\envs\Mayavi\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [14]:
ds_tsdata_emc3

<xarray.Dataset>
Dimensions:                       (delta_phi: 2, delta_r: 2, delta_theta: 2, phi: 36, r: 139, theta: 512, time: 20)
Coordinates:
    R_bounds                      (r, theta, phi, delta_r, delta_theta, delta_phi) float64 ...
    z_bounds                      (r, theta, phi, delta_r, delta_theta, delta_phi) float64 ...
    phi_bounds                    (phi, delta_phi) float64 ...
Dimensions without coordinates: delta_phi, delta_r, delta_theta, phi, r, theta, time
Data variables: (12/34)
    _plasma_map                   (r, theta, phi) int64 379649 379649 ... 398090
    ne                            (time, r, theta, phi) float64 nan nan ... nan
    nZ1                           (time, r, theta, phi) float64 nan nan ... nan
    nZ2                           (time, r, theta, phi) float64 nan nan ... nan
    nZ3                           (time, r, theta, phi) float64 nan nan ... nan
    nZ4                           (time, r, theta, phi) float64 nan nan ... nan
    ...                            ...
    IMPURITY_IONIZATION_SOURCE_0  (time, r, theta, phi) float64 nan nan ... nan
    IMPURITY_IONIZATION_SOURCE_1  (time, r, theta, phi) float64 nan nan ... nan
    IMPURITY_NEUTRAL_0            (time, r, theta, phi) float64 nan nan ... nan
    IMP_RADIATION_0               (time, r, theta, phi) float64 nan nan ... nan
    FLUX_CONSERVATION_0           (r, theta, phi) float64 nan nan ... nan nan
    LG_CELL_0                     (r, theta, phi) float64 nan nan ... nan nan

In [15]:
mean_sel

<xarray.Dataset>
Dimensions:                       (delta_phi: 2, delta_r: 2, delta_theta: 2, phi: 36, r: 139, theta: 512)
Coordinates:
    R_bounds                      (r, theta, phi, delta_r, delta_theta, delta_phi) float64 ...
    z_bounds                      (r, theta, phi, delta_r, delta_theta, delta_phi) float64 ...
    phi_bounds                    (phi, delta_phi) float64 0.0 ... 0.6283
Dimensions without coordinates: delta_phi, delta_r, delta_theta, phi, r, theta
Data variables: (12/34)
    _plasma_map                   (r, theta, phi) int64 379649 379649 ... 398090
    ne                            (r, theta, phi) float64 nan nan ... nan nan
    nZ1                           (r, theta, phi) float64 nan nan ... nan nan
    nZ2                           (r, theta, phi) float64 nan nan ... nan nan
    nZ3                           (r, theta, phi) float64 nan nan ... nan nan
    nZ4                           (r, theta, phi) float64 nan nan ... nan nan
    ...                            ...
    IMPURITY_IONIZATION_SOURCE_0  (r, theta, phi) float64 nan nan ... nan nan
    IMPURITY_IONIZATION_SOURCE_1  (r, theta, phi) float64 nan nan ... nan nan
    IMPURITY_NEUTRAL_0            (r, theta, phi) float64 nan nan ... nan nan
    IMP_RADIATION_0               (r, theta, phi) float64 nan nan ... nan nan
    FLUX_CONSERVATION_0           (r, theta, phi) float64 nan nan ... nan nan
    LG_CELL_0                     (r, theta, phi) float64 nan nan ... nan nan

In [17]:
# std_sel
std_tot

<xarray.Dataset>
Dimensions:                       ()
Data variables: (12/34)
    _plasma_map                   float64 1.833e+05
    ne                            float64 8.171e+18
    nZ1                           float64 1.733e+16
    nZ2                           float64 4.181e+16
    nZ3                           float64 7.049e+16
    nZ4                           float64 2.253e+17
    ...                            ...
    IMPURITY_IONIZATION_SOURCE_0  float64 0.00214
    IMPURITY_IONIZATION_SOURCE_1  float64 0.00214
    IMPURITY_NEUTRAL_0            float64 1.7e+10
    IMP_RADIATION_0               float64 0.2549
    FLUX_CONSERVATION_0           float64 0.1077
    LG_CELL_0                     float64 1.022